# **RA1: annual data**

Dong Gyun Ko <br/>
last updated: august 29, 2022 <br/>

In [97]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
# import libraries

import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader as pdr
import pandas_datareader.data as web
from pandas_datareader import wb
import requests # python 3.6

import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [99]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_data')

In [100]:
# set the pandas display option

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# pd.options.display.max_rows = 100
# pd.options.display.max_columns = 100

## **1. OECD**

### **1.1. python code**

In [101]:
# import the raw data

var_list = ['oecd_a_ggdebt', 'oecd_a_ggexp', 'oecd_a_ginv', 'oecd_a_gdp', 'oecd_a_gfcf', 'oecd_a_cpi', 'oecd_a_ca', 'oecd_a_stir']

for var in var_list:

    globals()['df_{}'.format(var)] = pd.read_csv('df_' + var + '.csv')
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].sort_values(by=['isocode', 'year'])

In [102]:
# merge the raw data 

df_oecd_a = pd.merge(globals()['df_{}'.format(var_list[0])], globals()['df_{}'.format(var_list[1])], how='outer', on=['isocode', 'year'])

for var in var_list[2:]:

    df_oecd_a = pd.merge(df_oecd_a, globals()['df_{}'.format(var)], how='outer', on=['isocode', 'year'])

In [103]:
# filter

# West Germany
cond0 = (df_oecd_a['isocode'] == 'DEW')

# countries group
cond1 = (df_oecd_a['isocode'] == 'EA')
cond2 = (df_oecd_a['isocode'] == 'EA19')
cond3 = (df_oecd_a['isocode'] == 'EU')
cond4 = (df_oecd_a['isocode'] == 'EU27_2020')
cond5 = (df_oecd_a['isocode'] == 'G-20')
cond6 = (df_oecd_a['isocode'] == 'G-7')
cond7 = (df_oecd_a['isocode'] == 'OAVG')
cond8 = (df_oecd_a['isocode'] == 'OECD')
cond9 = (df_oecd_a['isocode'] == 'OECDE')
cond10 = (df_oecd_a['isocode'] == 'EA19')

df_oecd_a = df_oecd_a.loc[~cond0 & ~cond1 & ~cond2 & ~cond3 & ~cond4 & ~cond5 & ~cond6 & ~cond7 & ~cond8 & ~cond9 & ~cond10]

In [104]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_oecd_a = pd.merge(df_wb_isocode, df_oecd_a, how='right', on=['isocode'])
df_oecd_a = df_oecd_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

In [105]:
# oecd annual dataset

df_oecd_a

,country,isocode,year,oecd_a_ggdebt,oecd_a_ggexp,oecd_a_ginv,oecd_a_gdp,oecd_a_gfcf,oecd_a_cpi,oecd_a_ca,oecd_a_stir
0,Albania,ALB,1997,NaN,NaN,NaN,8555.93,1721.06,NaN,NaN,NaN
1,Albania,ALB,1998,NaN,NaN,NaN,9451.51,2031.00,NaN,NaN,NaN
2,Albania,ALB,1999,NaN,NaN,NaN,10792.63,2540.92,NaN,NaN,NaN
3,Albania,ALB,2000,NaN,NaN,NaN,11926.20,3805.24,NaN,NaN,NaN
4,Albania,ALB,2001,NaN,NaN,NaN,13157.36,4832.61,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2995,Zambia,ZMB,2015,NaN,NaN,NaN,54472.60,20939.01,NaN,NaN,NaN
2996,Zambia,ZMB,2016,NaN,NaN,NaN,55712.21,20290.72,NaN,NaN,NaN
2997,Zambia,ZMB,2017,NaN,NaN,NaN,58735.19,22794.20,NaN,NaN,NaN
2998,Zambia,ZMB,2018,NaN,NaN,NaN,62564.66,21966.53,NaN,NaN,NaN


### **1.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|general government debt|oecd_a_ggdebt|% of GDP|annual|GGDEBT|TOT|PC_GDP|https://data.oecd.org/gga/general-government-debt.htm|
|2|general government spending|oecd_a_ggexp|% of GDP|annual|GGEXP|TOT|PC_GDP|https://data.oecd.org/gga/general-government-spending.htm|
|3|investment by sector(government)|oecd_a_ginv|% of GFCF|annual|GFCFSECTOR|GG|PC_GFCF|https://data.oecd.org/gdp/investment-by-sector.htm|
|4|gross domestic product|oecd_a_gdp|million USD|annual|GDP|TOT|MLN_USD|https://data.oecd.org/gdp/gross-domestic-product-gdp.htm|
|5|gross fixed capital formation|oecd_a_gfcf|million USD|annual|GFCF|TOT|MLN_USD|https://data.oecd.org/gdp/investment-gfcf.htm|
|6|consumer price index|oecd_a_cpi|2015=100|annual|CPI|TOT|IDX2015|https://data.oecd.org/price/inflation-cpi.htm|
|7|current account balance|oecd_a_ca|% of GDP|annual|BOP|TOT|PC_GDP|https://data.oecd.org/trade/current-account-balance.htm|
|8|short-term interest rates|oecd_a_stir|% per annum|annual|STINT|TOT|PC_PA|https://data.oecd.org/interest/short-term-interest-rates.htm|

### **1.3. data summary**

In [106]:
# number of countries & sample period

print('number of countries:', df_oecd_a['isocode'].unique().shape[0])
print('start:', np.min(df_oecd_a['year'].unique()))
print('end:', np.max(df_oecd_a['year'].unique()))

number of countries: 62
start: 1914
end: 2021


In [107]:
# non-null count by each variables

df_oecd_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   country        3000 non-null   object 
 1   isocode        3000 non-null   object 
 2   year           3000 non-null   int64  
 3   oecd_a_ggdebt  909 non-null    float64
 4   oecd_a_ggexp   864 non-null    float64
 5   oecd_a_ginv    1165 non-null   float64
 6   oecd_a_gdp     2483 non-null   float64
 7   oecd_a_gfcf    2423 non-null   float64
 8   oecd_a_cpi     2555 non-null   float64
 9   oecd_a_ca      1292 non-null   float64
 10  oecd_a_stir    1375 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 257.9+ KB


In [108]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_oecd_a.describe(), 2)

,year,oecd_a_ggdebt,oecd_a_ggexp,oecd_a_ginv,oecd_a_gdp,oecd_a_gfcf,oecd_a_cpi,oecd_a_ca,oecd_a_stir
count,3000.00,909.00,864.00,1165.00,2483.00,2423.00,2555.00,1292.00,1375.00
mean,1992.34,71.28,43.22,16.14,867966.38,218639.25,51.10,-0.09,6.10
std,20.62,42.35,8.49,4.36,2215186.64,680177.42,38.20,5.12,6.04
min,1914.00,6.65,17.90,2.58,786.78,225.73,0.00,-22.67,-0.82
25%,1978.00,43.25,37.92,13.08,61143.61,14062.83,12.00,-3.09,1.75
50%,1996.00,62.18,43.65,16.04,208937.27,47827.37,52.77,-0.55,4.63
75%,2009.00,92.84,49.30,18.72,694176.59,147838.01,86.26,2.66,8.68
max,2021.00,259.46,64.89,38.52,24313684.92,10305337.02,215.52,26.28,45.48


## **2. BIS**

### **2.1. python code** <br/>

In [109]:
# import the raw data

var_list = ['bis_a_ggdebt_m', 'bis_a_ggdebt_n', 'bis_a_cbpr', 'bis_a_reer_b', 'bis_a_reer_n']

for var in var_list:

    globals()['df_{}'.format(var)] = pd.read_csv('df_' + var + '.csv').set_index('isocode').transpose()
    globals()['df_{}'.format(var)] = pd.DataFrame(globals()['df_{}'.format(var)].stack(level='isocode')).reset_index()
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'level_0':'year', 0:var})
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)][['isocode', 'year', var]].sort_values(by=['isocode', 'year']).reset_index(drop=True)

In [110]:
# merge the raw data 

df_bis_a = pd.merge(globals()['df_{}'.format(var_list[0])], globals()['df_{}'.format(var_list[1])], how='outer', on=['isocode', 'year'])

for var in var_list[2:]:

    df_bis_a = pd.merge(df_bis_a, globals()['df_{}'.format(var)], how='outer', on=['isocode', 'year'])

In [111]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_bis_a = pd.merge(df_wb_isocode, df_bis_a, how='right', on=['isocode'])
df_bis_a = df_bis_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

In [112]:
# filter

# Taiwan
cond0 = (df_bis_a['isocode'] == 'TWN')

df_bis_a = df_bis_a.loc[~cond0]

In [113]:
# set the datetime

df_bis_a['year'] = pd.to_datetime(df_bis_a['year'])
df_bis_a['year'] = df_bis_a['year'].dt.year

In [114]:
# bis annual dataset

df_bis_a = df_bis_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

df_bis_a

,country,isocode,year,bis_a_ggdebt_m,bis_a_ggdebt_n,bis_a_cbpr,bis_a_reer_b,bis_a_reer_n
0,United Arab Emirates,ARE,1994,NaN,NaN,NaN,74.66,NaN
1,United Arab Emirates,ARE,1995,NaN,NaN,NaN,71.24,NaN
2,United Arab Emirates,ARE,1996,NaN,NaN,NaN,73.91,NaN
3,United Arab Emirates,ARE,1997,NaN,NaN,NaN,79.67,NaN
4,United Arab Emirates,ARE,1998,NaN,NaN,NaN,87.11,NaN
...,...,...,...,...,...,...,...,...
2684,South Africa,ZAF,2017,NaN,50.30,6.75,79.53,NaN
2685,South Africa,ZAF,2018,NaN,53.20,6.75,80.78,NaN
2686,South Africa,ZAF,2019,NaN,57.80,6.5,78.19,NaN
2687,South Africa,ZAF,2020,NaN,71.00,3.5,70.36,NaN


### **2.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|total credit to the government sector at market value (core debt)|bis_a_ggdebt_m|% of GDP|annual|CRE|-|market value|https://stats.bis.org/statx/srs/table/f5.1|
|2|total credit to the government sector at nominal value (core debt)|bis_a_ggdebt_n|% of GDP|annual|CRE|-|nominal value|https://stats.bis.org/statx/srs/table/f5.4|
|3|central bank policy rates|bis_a_cbpr|%|annual|CBP|-|-|https://stats.bis.org/statx/srs/table/l1|
|4|real effective exchange rate, broad(60 economies) indicies|bis_a_reer_b|2010=100|annual|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|5|real effective exchange rate, narrow(27 economies) indicies|bis_a_reer_n|2010=100|annual|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|


### **2.3. data summary**

In [115]:
# number of countries & sample period

print('number of countries:', df_bis_a['isocode'].unique().shape[0])
print('start:', np.min(df_bis_a['year'].unique()))
print('end:', np.max(df_bis_a['year'].unique()))

number of countries: 60
start: 1946
end: 2021


In [116]:
# non-null count by each variables

df_bis_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2689 entries, 0 to 2688
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         2689 non-null   object 
 1   isocode         2689 non-null   object 
 2   year            2689 non-null   int64  
 3   bis_a_ggdebt_m  812 non-null    float64
 4   bis_a_ggdebt_n  1219 non-null   float64
 5   bis_a_cbpr      1393 non-null   object 
 6   bis_a_reer_b    1624 non-null   float64
 7   bis_a_reer_n    1450 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 168.2+ KB


In [117]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_bis_a.describe(), 2)

,year,bis_a_ggdebt_m,bis_a_ggdebt_n,bis_a_reer_b,bis_a_reer_n
count,2689.00,812.00,1219.00,1624.00,1450.00
mean,1995.55,65.56,59.21,97.29,100.17
std,18.12,39.56,34.44,17.79,17.04
min,1946.00,4.60,1.60,43.41,47.52
25%,1982.00,39.38,34.25,90.54,90.78
50%,1999.00,58.15,51.70,98.11,98.63
75%,2010.00,84.80,74.25,103.19,106.79
max,2021.00,238.20,226.90,276.38,171.51


## **3. World Bank**

### **3.1. python code** <br/>

In [118]:
# import the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8')

wb_isocode_list = []

for i in range(0,df_wb_isocode['alpha-3'].unique().shape[0]):

    alpha_3 = str(df_wb_isocode['alpha-3'].unique()[i])
    wb_isocode_list.append(alpha_3)

In [119]:
# debug HTTPConnectionPool error

from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

retries = Retry(connect=5, read=3, redirect=3)
http_session = requests.Session()
http_session.mount('https://<yourdomain>.slack.com', HTTPAdapter(max_retries=retries))

In [120]:
# import the raw data

indicator_list = ['NE.CON.GOVT.ZS',
                  'NY.GDP.MKTP.CD',
                  'NY.GDP.MKTP.KD',
                  'NE.GDI.FTOT.CD',
                  'FP.CPI.TOTL',
                  'NY.GDP.DEFL.KD.ZG',
                  'BN.CAB.XOKA.GD.ZS',
                  'PX.REX.REER']

var_list = ['wb_a_ggexp', 'wb_a_ngdp', 'wb_a_rgpd', 'wb_a_gfcf', 'wb_a_cpi', 'wb_a_gdpd',  'wb_a_ca', 'wb_a_reer']

for i in range(0,len(indicator_list)):

    globals()['df_{}'.format(var_list[i])] = wb.download(indicator=indicator_list[i], country=wb_isocode_list, start=1960, end=2021)
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])].reset_index()
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])].rename(columns={'country':'country', 'year':'year', indicator_list[i]:var_list[i]})
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])][['country', 'year', var_list[i]]].sort_values(by=['country', 'year']).reset_index(drop=True)
    
    # merge the wb isocode data
    globals()['df_{}'.format(var_list[i])] = pd.merge(globals()['df_{}'.format(var_list[i])], df_wb_isocode, how='outer', on=['country']).rename(columns={'alpha-3':'isocode'})
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])][['country', 'isocode', 'year',  var_list[i]]]

/usr/local/lib/python3.7/dist-packages/pandas_datareader/wb.py:593: UserWarning: Non-standard ISO country codes: CHI, XKX
  "Non-standard ISO " "country codes: %s" % tmp, UserWarning


In [121]:
# merge the raw data 

df_wb_a = pd.merge(globals()['df_{}'.format(var_list[0])], globals()['df_{}'.format(var_list[1])], how='outer', on=['country', 'isocode', 'year'])

for var in var_list[2:]:

    df_wb_a = pd.merge(df_wb_a, globals()['df_{}'.format(var)], how='outer', on=['country', 'isocode', 'year'])

In [122]:
# set the datetime

df_wb_a['year'] = pd.to_datetime(df_wb_a['year'])
df_wb_a['year'] = df_wb_a['year'].dt.year

In [123]:
# world bank dataset

df_wb_a = df_wb_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

df_wb_a

,country,isocode,year,wb_a_ggexp,wb_a_ngdp,wb_a_rgpd,wb_a_gfcf,wb_a_cpi,wb_a_gdpd,wb_a_ca,wb_a_reer
0,Aruba,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,ABW,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13449,Zimbabwe,ZWE,2017,21.65,17584890936.65,21061283685.95,1699377692.68,105.51,2.44,-1.54,NaN
13450,Zimbabwe,ZWE,2018,11.92,18115543790.79,22077324353.29,1751060497.52,116.71,59.80,-7.62,NaN
13451,Zimbabwe,ZWE,2019,6.64,19284289739.05,20720841373.13,1427892358.82,414.68,440.83,4.77,NaN
13452,Zimbabwe,ZWE,2020,7.76,18051170798.94,19426048165.88,1344952699.72,2725.31,558.56,6.07,NaN


### **3.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|general government final consumption expenditure|wb_a_ggexp|% of GDP|annual|NE.CON.GOVT.ZS|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NE.CON.GOVT.ZS&country=|
|2|GDP (current USD)|wb_a_ngdp|current USD|annual|NY.GDP.MKTP.CD|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=|
|3|GDP (constant 2015 USD)|wb_a_rgdp|constant 2015 USD|annual|NY.GDP.MKTP.KD|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.KD&country=|
|4|gross fixed capital formation (current USD)|wb_a_gfcf|current USD|annual|NE.GDI.FTOT.CD|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NE.GDI.FTOT.CD&country=|
|5|consumer price index (2010=100)|wb_a_cpi|2010=100|annual|FP.CPI.TOTL|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=FP.CPI.TOTL&country=|
|6|gdp deflator|wb_a_gdpd|annual %|annual|NY.GDP.DEFL.KD.ZG|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.DEFL.KD.ZG&country=|
|7|current account balance|wb_a_ca|% of GDP|annual|BN.CAB.XOKA.GD.ZS|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=BN.CAB.XOKA.GD.ZS&country=|
|8|real effective exchange rate index (2010=100)|wb_a_reer|2010=100|annual|PX.REX.REER|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=PX.REX.REER&country=|

### **3.3. data summary** <br/>

In [124]:
# number of countries & sample period

print('number of countries:', df_wb_a['isocode'].unique().shape[0])
print('start:', np.min(df_wb_a['year'].unique()))
print('end:', np.max(df_wb_a['year'].unique()))

number of countries: 217
start: 1960
end: 2021


In [125]:
# non-null count by each variables

df_wb_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13454 entries, 0 to 13453
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     13454 non-null  object 
 1   isocode     13454 non-null  object 
 2   year        13454 non-null  int64  
 3   wb_a_ggexp  8143 non-null   float64
 4   wb_a_ngdp   10336 non-null  float64
 5   wb_a_rgpd   9791 non-null   float64
 6   wb_a_gfcf   7587 non-null   float64
 7   wb_a_cpi    8463 non-null   float64
 8   wb_a_gdpd   9784 non-null   float64
 9   wb_a_ca     6864 non-null   float64
 10  wb_a_reer   3779 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 1.1+ MB


In [126]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_wb_a.describe(), 2)

,year,wb_a_ggexp,wb_a_ngdp,wb_a_rgpd,wb_a_gfcf,wb_a_cpi,wb_a_gdpd,wb_a_ca,wb_a_reer
count,13454.00,8143.00,10336.00,9791.00,7587.00,8463.00,9784.00,6864.00,3779.00
mean,1990.50,16.28,184192121208.18,246060250890.96,58498628436.55,75.96,29.58,-2.93,115.02
std,17.90,8.17,991080605921.11,1123327823642.51,288038289788.06,435.00,407.00,13.07,105.61
min,1960.00,0.00,8824447.74,21561952.31,-20612328.07,0.00,-98.70,-240.52,18.73
25%,1975.00,11.20,1460824271.16,3652104951.13,695080308.53,16.46,1.92,-7.12,93.33
50%,1990.50,15.31,7824737791.80,15610486135.64,3386606655.81,61.54,5.06,-2.89,100.56
75%,2006.00,19.42,52326788295.34,100386043670.39,22026083272.61,100.00,11.04,0.98,112.92
max,2021.00,147.72,22996100000000.00,20338578000000.00,6240217076281.01,22570.71,26765.86,311.76,3053.70


## **4. pooled data**

### **4.1. python code** <br/>

In [127]:
# generate the annual pooled data

df_oecd_bis_a = pd.merge(df_oecd_a, df_bis_a, how='outer', on=['country', 'isocode', 'year'])
df_a_pooled = pd.merge(df_oecd_bis_a, df_wb_a, how='outer', on=['country', 'isocode', 'year'])
df_a_pooled = df_a_pooled.sort_values(by=['isocode', 'year']).reset_index(drop=True)

In [128]:
# generate the annual temp data

df_a_temp = pd.read_csv('df_a_temp.csv').set_index(['country', 'alpha-3']).transpose()
df_a_temp = pd.DataFrame(df_a_temp.stack(level=['country', 'alpha-3'])).reset_index()
df_a_temp = df_a_temp.rename(columns={'level_0':'year', 'alpha-3':'isocode', 0:'value'})
df_a_temp = df_a_temp[['country', 'isocode', 'year', 'value']].sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_a_temp['year'] = pd.to_datetime(df_a_temp['year']).dt.year

In [129]:
# merge the annual pooled data with the annual temp data

df_a_pooled = pd.merge(df_a_pooled, df_a_temp, how='outer', on=['country', 'isocode', 'year']).drop(columns=['value'])

In [130]:
# annual pooled dataset

df_a_pooled = df_a_pooled.sort_values(by=['isocode', 'year']).reset_index(drop=True)

df_a_pooled

,country,isocode,year,oecd_a_ggdebt,oecd_a_ggexp,oecd_a_ginv,oecd_a_gdp,oecd_a_gfcf,oecd_a_cpi,oecd_a_ca,...,bis_a_reer_b,bis_a_reer_n,wb_a_ggexp,wb_a_ngdp,wb_a_rgpd,wb_a_gfcf,wb_a_cpi,wb_a_gdpd,wb_a_ca,wb_a_reer
0,Aruba,ABW,1914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,1915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,ABW,1916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,1917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,1918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23431,Zimbabwe,ZWE,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,21.65,17584890936.65,21061283685.95,1699377692.68,105.51,2.44,-1.54,NaN
23432,Zimbabwe,ZWE,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.92,18115543790.79,22077324353.29,1751060497.52,116.71,59.80,-7.62,NaN
23433,Zimbabwe,ZWE,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.64,19284289739.05,20720841373.13,1427892358.82,414.68,440.83,4.77,NaN
23434,Zimbabwe,ZWE,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.76,18051170798.94,19426048165.88,1344952699.72,2725.31,558.56,6.07,NaN


### **4.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|$OECD$|||||||||
|1|general government debt|oecd_a_ggdebt|% of GDP|annual|GGDEBT|TOT|PC_GDP|https://data.oecd.org/gga/general-government-debt.htm|
|2|general government spending|oecd_a_ggexp|% of GDP|annual|GGEXP|TOT|PC_GDP|https://data.oecd.org/gga/general-government-spending.htm|
|3|investment by sector(government)|oecd_a_ginv|% of GFCF|annual|GFCFSECTOR|GG|PC_GFCF|https://data.oecd.org/gdp/investment-by-sector.htm|
|4|gross domestic product|oecd_a_gdp|million USD|annual|GDP|TOT|MLN_USD|https://data.oecd.org/gdp/gross-domestic-product-gdp.htm|
|5|gross fixed capital formation|oecd_a_gfcf|million USD|annual|GFCF|TOT|MLN_USD|https://data.oecd.org/gdp/investment-gfcf.htm|
|6|consumer price index|oecd_a_cpi|2015=100|annual|CPI|TOT|IDX2015|https://data.oecd.org/price/inflation-cpi.htm|
|7|current account balance|oecd_a_ca|% of GDP|annual|BOP|TOT|PC_GDP|https://data.oecd.org/trade/current-account-balance.htm|
|8|short-term interest rates|oecd_a_stir|% per annum|annual|STINT|TOT|PC_PA|https://data.oecd.org/interest/short-term-interest-rates.htm|
|$BIS$|||||||||
|1|total credit to the government sector at market value (core debt)|bis_a_ggdebt_m|% of GDP|annual|CRE|-|market value|https://stats.bis.org/statx/srs/table/f5.1|
|2|total credit to the government sector at nominal value (core debt)|bis_a_ggdebt_n|% of GDP|annual|CRE|-|nominal value|https://stats.bis.org/statx/srs/table/f5.4|
|3|central bank policy rates|bis_a_cbpr|%|annual|CBP|-|-|https://stats.bis.org/statx/srs/table/l1|
|4|real effective exchange rate, broad(60 economies) indicies|bis_a_reer_b|2010=100|annual|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|5|real effective exchange rate, narrow(27 economies) indicies|bis_a_reer_n|2010=100|annual|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|$World$ $Bank$|||||||||
|1|general government final consumption expenditure|wb_a_ggexp|% of GDP|annual|NE.CON.GOVT.ZS|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NE.CON.GOVT.ZS&country=|
|2|GDP (current USD)|wb_a_ngdp|current USD|annual|NY.GDP.MKTP.CD|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=|
|3|GDP (constant 2015 USD)|wb_a_rgdp|constant 2015 USD|annual|NY.GDP.MKTP.KD|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.KD&country=|
|4|gross fixed capital formation (current USD)|wb_a_gfcf|current USD|annual|NE.GDI.FTOT.CD|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NE.GDI.FTOT.CD&country=|
|5|consumer price index (2010=100)|wb_a_cpi|2010=100|annual|FP.CPI.TOTL|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=FP.CPI.TOTL&country=|
|6|gdp deflator|wb_a_gdpd|annual %|annual|NY.GDP.DEFL.KD.ZG|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.DEFL.KD.ZG&country=|
|7|current account balance|wb_a_ca|% of GDP|annual|BN.CAB.XOKA.GD.ZS|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=BN.CAB.XOKA.GD.ZS&country=|
|8|real effective exchange rate index (2010=100)|wb_a_reer|2010=100|annual|PX.REX.REER|-|-|https://databank.worldbank.org/reports.aspx?source=2&series=PX.REX.REER&country=|

### **4.3. data summary**

In [131]:
# number of countries & sample period

print('number of countries:', df_a_pooled['isocode'].unique().shape[0])
print('sample period: from 1914 to 2021, annual')

number of countries: 217
sample period: from 1914 to 2021, annual


In [132]:
# non-null count by each variables

df_a_pooled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23436 entries, 0 to 23435
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         23436 non-null  object 
 1   isocode         23436 non-null  object 
 2   year            23436 non-null  int64  
 3   oecd_a_ggdebt   909 non-null    float64
 4   oecd_a_ggexp    864 non-null    float64
 5   oecd_a_ginv     1165 non-null   float64
 6   oecd_a_gdp      2483 non-null   float64
 7   oecd_a_gfcf     2423 non-null   float64
 8   oecd_a_cpi      2555 non-null   float64
 9   oecd_a_ca       1292 non-null   float64
 10  oecd_a_stir     1375 non-null   float64
 11  bis_a_ggdebt_m  812 non-null    float64
 12  bis_a_ggdebt_n  1219 non-null   float64
 13  bis_a_cbpr      1393 non-null   object 
 14  bis_a_reer_b    1624 non-null   float64
 15  bis_a_reer_n    1450 non-null   float64
 16  wb_a_ggexp      8143 non-null   float64
 17  wb_a_ngdp       10336 non-null 

In [133]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_a_pooled.describe(), 2)

,year,oecd_a_ggdebt,oecd_a_ggexp,oecd_a_ginv,oecd_a_gdp,oecd_a_gfcf,oecd_a_cpi,oecd_a_ca,oecd_a_stir,bis_a_ggdebt_m,...,bis_a_reer_b,bis_a_reer_n,wb_a_ggexp,wb_a_ngdp,wb_a_rgpd,wb_a_gfcf,wb_a_cpi,wb_a_gdpd,wb_a_ca,wb_a_reer
count,23436.00,909.00,864.00,1165.00,2483.00,2423.00,2555.00,1292.00,1375.00,812.00,...,1624.00,1450.00,8143.00,10336.00,9791.00,7587.00,8463.00,9784.00,6864.00,3779.00
mean,1967.50,71.28,43.22,16.14,867966.38,218639.25,51.10,-0.09,6.10,65.56,...,97.29,100.17,16.28,184192121208.18,246060250890.96,58498628436.55,75.96,29.58,-2.93,115.02
std,31.18,42.35,8.49,4.36,2215186.64,680177.42,38.20,5.12,6.04,39.56,...,17.79,17.04,8.17,991080605921.11,1123327823642.51,288038289788.06,435.00,407.00,13.07,105.61
min,1914.00,6.65,17.90,2.58,786.78,225.73,0.00,-22.67,-0.82,4.60,...,43.41,47.52,0.00,8824447.74,21561952.31,-20612328.07,0.00,-98.70,-240.52,18.73
25%,1940.75,43.25,37.92,13.08,61143.61,14062.83,12.00,-3.09,1.75,39.38,...,90.54,90.78,11.20,1460824271.16,3652104951.13,695080308.53,16.46,1.92,-7.12,93.33
50%,1967.50,62.18,43.65,16.04,208937.27,47827.37,52.77,-0.55,4.63,58.15,...,98.11,98.63,15.31,7824737791.80,15610486135.64,3386606655.81,61.54,5.06,-2.89,100.56
75%,1994.25,92.84,49.30,18.72,694176.59,147838.01,86.26,2.66,8.68,84.80,...,103.19,106.79,19.42,52326788295.34,100386043670.39,22026083272.61,100.00,11.04,0.98,112.92
max,2021.00,259.46,64.89,38.52,24313684.92,10305337.02,215.52,26.28,45.48,238.20,...,276.38,171.51,147.72,22996100000000.00,20338578000000.00,6240217076281.01,22570.71,26765.86,311.76,3053.70


## **5. panel data**

### **5.1. python code** <br/>

In [136]:
# generate the count matrix

count_N_df_a_pooled = df_a_pooled.groupby(df_a_pooled['isocode']).count()
count_T_df_a_pooled = df_a_pooled.groupby(df_a_pooled['year']).count()

In [138]:
# filter (N)

# ggdebt
condN1 = (count_N_df_a_pooled['oecd_a_ggdebt'] == 0)
condN2 = (count_N_df_a_pooled['bis_a_ggdebt_m'] == 0)
condN3 = (count_N_df_a_pooled['bis_a_ggdebt_n'] == 0)

# ggexp
condN4 = (count_N_df_a_pooled['oecd_a_ggexp'] == 0)
condN5 = (count_N_df_a_pooled['wb_a_ggexp'] == 0)

count_N_df_a_pooled = count_N_df_a_pooled.loc[(~condN1 | ~condN2 | ~condN3) & (~condN4 | ~condN5)]
count_N_df_a_pooled = count_N_df_a_pooled.reset_index()

print('countries:', count_N_df_a_pooled['isocode'].unique().shape[0])

countries: 50


In [140]:
# filter (T)

# ggdebt
condT1 = (count_T_df_a_pooled['oecd_a_ggdebt'] == 0)
condT2 = (count_T_df_a_pooled['bis_a_ggdebt_m'] == 0)
condT3 = (count_T_df_a_pooled['bis_a_ggdebt_n'] == 0)

# ggexp
condT4 = (count_T_df_a_pooled['oecd_a_ggexp'] == 0)
condT5 = (count_T_df_a_pooled['wb_a_ggexp'] == 0)

count_T_df_a_pooled = count_T_df_a_pooled.loc[(~condT1 | ~condT2 | ~condT3) & (~condT4 | ~condT5)]
count_T_df_a_pooled = count_T_df_a_pooled.reset_index()

print('start:', count_T_df_a_pooled['year'].unique().min())

start: 1960


In [141]:
# generate the panel data

# select the T
cond_T = (df_a_pooled['year'] >= 1960)

df_a_panel = df_a_pooled.loc[cond_T]

# select the N
print(count_N_df_a_pooled['isocode'].unique().shape[0])
isocode_list = count_N_df_a_pooled['isocode'].unique()

for i in range(0, count_N_df_a_pooled['isocode'].unique().shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_a_pooled['isocode'] == isocode_list[i])

df_a_panel = df_a_panel.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

50


In [142]:
# annual panel data

df_a_panel = df_a_panel.sort_values(by=['isocode', 'year']).reset_index(drop=True)

df_a_panel

,country,isocode,year,oecd_a_ggdebt,oecd_a_ggexp,oecd_a_ginv,oecd_a_gdp,oecd_a_gfcf,oecd_a_cpi,oecd_a_ca,...,bis_a_reer_b,bis_a_reer_n,wb_a_ggexp,wb_a_ngdp,wb_a_rgpd,wb_a_gfcf,wb_a_cpi,wb_a_gdpd,wb_a_ca,wb_a_reer
0,Argentina,ARG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.88,NaN,150797810295.88,NaN,NaN,NaN,NaN,NaN
1,Argentina,ARG,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.79,NaN,158982878499.52,NaN,NaN,20.31,NaN,NaN
2,Argentina,ARG,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.19,24450604877.49,157628310158.57,5313239494.55,NaN,28.87,NaN,NaN
3,Argentina,ARG,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.25,18272123664.40,149261089202.81,3338776681.38,NaN,25.59,NaN,NaN
4,Argentina,ARG,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.23,25605249381.76,164381681831.92,4398700136.20,NaN,28.77,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,South Africa,ZAF,2017,NaN,NaN,16.84,724100.73,135874.19,112.10,-2.37,...,79.53,NaN,19.24,381448814653.46,353055253707.07,62553047976.02,146.05,5.47,-2.37,79.38
3096,South Africa,ZAF,2018,NaN,NaN,16.43,746947.78,135849.18,117.16,-2.94,...,80.78,NaN,19.37,404842116738.07,358307364996.50,64168364125.47,152.65,3.96,-3.04,80.73
3097,South Africa,ZAF,2019,NaN,NaN,15.11,757983.88,135676.54,121.99,-2.57,...,78.19,NaN,19.61,387934574098.17,358712444718.33,59535161173.56,158.94,4.50,-2.59,78.16
3098,South Africa,ZAF,2020,NaN,NaN,NaN,713818.72,112457.91,125.90,1.97,...,70.36,NaN,20.65,335442101366.42,335640150576.12,46012135687.86,164.04,5.27,2.03,70.93


### **5.2. data summary**

In [143]:
# number of countries & sample period

print('number of countries:', df_a_panel['isocode'].unique().shape[0])
print(df_a_panel['isocode'].unique())
print('start:', np.min(df_a_panel['year'].unique()))
print('end:', np.max(df_a_panel['year'].unique()))

number of countries: 50
['ARG' 'AUS' 'AUT' 'BEL' 'BRA' 'CAN' 'CHE' 'CHL' 'CHN' 'COL' 'CZE' 'DEU'
 'DNK' 'ESP' 'EST' 'FIN' 'FRA' 'GBR' 'GRC' 'HKG' 'HUN' 'IDN' 'IND' 'IRL'
 'ISL' 'ISR' 'ITA' 'JPN' 'KOR' 'LTU' 'LUX' 'LVA' 'MEX' 'MYS' 'NLD' 'NOR'
 'NZL' 'POL' 'PRT' 'ROU' 'RUS' 'SAU' 'SGP' 'SVK' 'SVN' 'SWE' 'THA' 'TUR'
 'USA' 'ZAF']
start: 1960
end: 2021


In [144]:
# non-null count by each variables

df_a_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3100 entries, 0 to 3099
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         3100 non-null   object 
 1   isocode         3100 non-null   object 
 2   year            3100 non-null   int64  
 3   oecd_a_ggdebt   909 non-null    float64
 4   oecd_a_ggexp    856 non-null    float64
 5   oecd_a_ginv     1159 non-null   float64
 6   oecd_a_gdp      2175 non-null   float64
 7   oecd_a_gfcf     2121 non-null   float64
 8   oecd_a_cpi      2306 non-null   float64
 9   oecd_a_ca       1264 non-null   float64
 10  oecd_a_stir     1361 non-null   float64
 11  bis_a_ggdebt_m  799 non-null    float64
 12  bis_a_ggdebt_n  1206 non-null   float64
 13  bis_a_cbpr      1172 non-null   object 
 14  bis_a_reer_b    1400 non-null   float64
 15  bis_a_reer_n    1450 non-null   float64
 16  wb_a_ggexp      2545 non-null   float64
 17  wb_a_ngdp       2715 non-null   f

In [145]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_a_panel.describe(), 2)

,year,oecd_a_ggdebt,oecd_a_ggexp,oecd_a_ginv,oecd_a_gdp,oecd_a_gfcf,oecd_a_cpi,oecd_a_ca,oecd_a_stir,bis_a_ggdebt_m,...,bis_a_reer_b,bis_a_reer_n,wb_a_ggexp,wb_a_ngdp,wb_a_rgpd,wb_a_gfcf,wb_a_cpi,wb_a_gdpd,wb_a_ca,wb_a_reer
count,3100.00,909.00,856.00,1159.00,2175.00,2121.00,2306.00,1264.00,1361.00,799.00,...,1400.00,1450.00,2545.00,2715.00,2605.00,2554.00,2698.00,2576.00,2018.00,1684.00
mean,1990.50,71.28,43.32,16.16,982695.36,247901.82,55.23,-0.02,6.11,65.48,...,97.44,100.17,16.53,647102498026.01,853562438141.25,162450973397.24,60.00,17.83,0.01,101.01
std,17.90,42.35,8.46,4.36,2344291.08,722247.93,37.27,5.14,6.06,39.84,...,18.55,17.04,5.10,1855493774541.71,2057539589992.97,479445544067.43,42.27,122.97,5.87,21.60
min,1960.00,6.65,17.90,2.58,786.78,225.73,0.00,-22.67,-0.82,4.60,...,43.41,47.52,2.98,248434096.97,5946828629.62,66934535.48,0.00,-98.70,-22.94,42.10
25%,1975.00,43.25,38.13,13.14,89139.84,20985.12,17.60,-2.96,1.73,38.75,...,90.12,90.78,12.15,35776908639.85,115076156531.86,9802683518.45,19.12,1.85,-3.19,91.62
50%,1990.50,62.18,43.79,16.07,275998.47,58755.83,59.21,-0.48,4.63,57.50,...,98.36,98.63,17.08,146775498080.00,254739721635.94,36413458627.34,63.28,4.20,-0.68,99.30
75%,2006.00,92.84,49.32,18.73,865996.89,188980.53,89.53,2.78,8.75,84.85,...,103.39,106.79,19.96,439365789361.19,712134720319.54,107520502640.97,96.78,8.89,2.53,106.32
max,2021.00,259.46,64.89,38.52,24313684.92,10305337.02,215.52,26.28,45.48,238.20,...,276.38,171.51,35.22,22996100000000.00,20338578000000.00,6240217076281.01,314.81,3046.09,50.70,269.74


## **6. export the panel data**

In [146]:
# annual panel data

# count_N_df_a_pooled.to_excel(excel_writer='count_N_df_a_pooled.xlsx')
# count_T_df_a_pooled.to_excel(excel_writer='count_T_df_a_pooled.xlsx')
# df_a_pooled.to_excel(excel_writer='df_a_pooled.xlsx')
df_a_panel.to_excel(excel_writer='df_a_panel.xlsx')

## **7. references**

* https://data.oecd.org/ <br/>
* https://stats.bis.org/#ppq=XRU_D_24D;pv=1,2~4~1,0,0~both <br/>
* https://databank.worldbank.org/source/world-development-indicators/preview/on <br/>
* https://wits.worldbank.org/wits/wits/witshelp/content/codes/country_codes.htm <br/>
* https://ec.europa.eu/eurostat/en/web/main/data/database <br/>
* https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1409151240976 <br/>